In [ ]:
from csv2anndata import *

In [ ]:
scATACS = run()

In [ ]:

def save_as_separate_anndata(adata: anndata.AnnData, output_dir: Path)->None:

    # Because of h5 limitations it is not possible to save an anndata object with
    # obsm/varm/uns DataFrames that have many columns (which is the case for e.g. gene_scores)
    # Thus we agreed, that we would seperate each matrix and save it as separate files.

    adatas = {}

    #peak_bc
    X = adata.X
    obs = adata.obs.copy()
    var = adata.var.copy()
    adatas['peak_bc'] = anndata.AnnData(X = X, obs = obs, var = var)

    #LSI_embedding
    X = adata.obsm['LSI_embedding'].values
    obs = adata.obs.copy()
    compute_ns(X, obs, 1)
    var = pd.DataFrame(data = None, index = adata.obsm['LSI_embedding'].columns)
    compute_ns(X, var, 0)
    adatas['LSI_embedding'] = anndata.AnnData(X = X, obs = obs, var = var)

    #gene_scores
    X = adata.obsm['gene_scores'].sparse.to_coo()
    X = X.tocsr() # anndata does not support coo when writing to h5.
    obs = adata.obs.copy()
    compute_ns(X, obs, 1)
    var = pd.DataFrame(data = None, index = adata.obsm['gene_scores'].columns)
    compute_ns(X, var, 0)
    adatas['gene_scores'] = anndata.AnnData(X = X, obs = obs, var = var)

    #markerpeak_target
    X = adata.uns['markerpeak_target'].values
    obs = pd.DataFrame(data = None, index = adata.uns['markerpeak_target'].index)
    compute_ns(X, obs, 1)
    var = pd.DataFrame(data = None, index = adata.uns['markerpeak_target'].columns)
    compute_ns(X, var, 0)
    adatas['markerpeak_target'] = anndata.AnnData(X = X, obs = obs, var = var)


    #ChromVar
    X = adata.obsm['ChromVar'].values
    obs = adata.obs.copy()
    compute_ns(X, obs, 1)
    var = pd.DataFrame(data = None, index = adata.obsm['ChromVar'].columns)
    compute_ns(X, var, 0)
    adatas['ChromVar'] = anndata.AnnData(X = X, obs = obs, var = var)

    for name, adata in adatas.items():
        utils.write_as_singles(adata, output_dir, name, add_h5=True)

    return adatas

In [ ]:
adatas = save_as_separate_anndata(scATACS, '')